# <center>NLP</center>

Посмотрим на данные и сделаем импорты необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
train = pd.read_csv('train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,"Gaelic translation \n\nHi, don't suppose you c...",0,0,0,0,0,0
1,1,Hey dick \nYou don't know what is copyright so...,1,0,1,0,1,0
2,2,"""\n\nAm I correct in thinking that you are ref...",0,0,0,0,0,0
3,3,== Notable Alumni == \n\n Why was this section...,0,0,0,0,0,0
4,4,I have already been sent this message about va...,1,0,1,0,1,0


In [3]:
train.shape

(150000, 8)

In [4]:
train.comment_text[1]

"Hey dick \nYou don't know what is copyright so shutup and get out of here, don't write stupid bullshit on my talk page. Dhivehi language is Dhivehi language not your bullshit nonsese."

Избавимся от всякого хлама)

In [5]:
train.comment_text = train.comment_text.apply(lambda x: re.sub('[!@#$;:%-&?<>*)(=_*/+|}{.,]','',x))
train.comment_text = train.comment_text.apply(lambda x: re.sub('\d+','',x))

In [6]:
train.comment_text[3]

' Notable Alumni  \n\n Why was this section all-but-deleted US has many notable allumni but currently the only one listed is a teacher who just happened to pitch for the Detroit Tigers a long long time ago in a galaxy far far away For one his son Michael Seelbach currently has a lead role in the Chicago production of Wicked I understand that it was often vandalized by students of the school but does that merit the almost complete deletion of it ps it looks like it was deleted a l-o-o-o-o-ng time ago'

Далее сделаем токенизацию и лемматизацию вручную, с помощью `nltk`

In [7]:
nltk.download('wordnet')

lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

train['new_comment'] = train.comment_text.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /home/ilya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


То же самое для тестовой выборки

In [8]:
test = pd.read_csv('test.csv')
test.comment_text = test.comment_text.apply(lambda x: re.sub('[!@#$;:%-&?<>*)(=_*/+}{|.,]','',x))
test.comment_text = test.comment_text.apply(lambda x: re.sub('\d+','',x))
# test['new_comment'] = test.comment_text.apply(lemmatize_text)

Векторизуем тексты, с помощью `tf-ifd`. Также здесь избавимся от стопслов и сделаем токенизацию дополнительно.
Вообще токенизацию с помощью nltk можно было не делать, так как она производится тут, но я попробовал несколько вариантов. И делать токенизацию с лемматизацией с помощью nltk оказалось эффективнее, поэтому тут закомментируем

In [10]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,
                                   strip_accents='unicode', 
                                   stop_words='english', 
                                  analyzer='word',
#                                   token_pattern=r'\w{1,}',
                                  ngram_range=(1,1),
                                   max_features=10000,
                          )

Для векторизации требуется, чтобы на входе был лист из строк, а у нас лист из листов, поэтому...

In [11]:
train['new_comment'] = [str(x) for x in train.comment_text]
test['new_comment'] = [str(x) for x in test.comment_text]

In [12]:
train.new_comment = train.new_comment.apply(lambda x: x.replace('\n', ''))
test.new_comment = test.new_comment.apply(lambda x: x.replace('\n', ''))

In [13]:
all_text = pd.concat([train.new_comment, test.new_comment])

In [14]:
alls = tfidf_vectorizer.fit(all_text)

In [15]:
alls

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
train_features = tfidf_vectorizer.transform(train.new_comment)
test_features = tfidf_vectorizer.transform(test.new_comment)

Сделали веторизацию текста, теперь сделаем векторизацию символов в тексте

In [17]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

char векторизацию здесь запускать не стал, тк слишком ресурсозатратная, а времени уже в обрез)

In [ ]:
%%time
char_vectorizer.fit(all_text)

In [ ]:
train_char_features = char_vectorizer.transform(train.new_comment)
test_char_features = char_vectorizer.transform(test.new_comment)

In [ ]:
from scipy.sparse import hstack
train_fin_features = hstack([train_char_features, train_features])

In [ ]:
test_fin_features = hstack([test_char_features, test_features])

Выделим лейблы для обучения модели

In [18]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Создадим отдельный дф для сабмита

In [19]:
sub = pd.DataFrame()
sub['id'] = test.id
for i in categories:
    sub[i] = 0

Теперь самое интересное) Наверное, самое разумное сразу подумать об обучении с учителем. Пройдемся по тем методам, которые очень хорошо работают с вероятностью. В первую очередь: `Logistig Regression` и `Random Forest`

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.1, solver='sag')
for category in categories:
    print('... Processing {}'.format(category))
    model.fit(train_features, train[category])
    prediction = model.predict_proba(test_features)[:,1]
    sub[category] = prediction

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate


In [22]:
train_features

<150000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3055756 stored elements in Compressed Sparse Row format>

'toxic'

In [25]:
from sklearn.model_selection import cross_val_score
scores = []
for train_target in categories:
    cv_score = np.mean(cross_val_score(model, train_features, train[train_target], cv=3, scoring='roc_auc')) 
    scores.append(cv_score)
np.mean(scores)

0.9688289687112009

In [26]:
sub.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,150000,0.278040,0.006556,0.051493,0.003417,0.098566,0.007584
1,150001,0.034944,0.005216,0.023202,0.002938,0.018518,0.006263
2,150002,0.038319,0.004483,0.021228,0.002466,0.020202,0.004869
3,150003,0.008351,0.003556,0.008114,0.002175,0.007408,0.003989
4,150004,0.059174,0.004122,0.027986,0.002476,0.027352,0.006226


In [27]:
sub.to_csv('sub.csv', index=False)

Параметры для модели подобрал вручную. Получается очень неплохая точность, однако этого не достаточно для попадания в топ)

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rb = RandomForestClassifier()
for category in categories:
    print('... Processing {}'.format(category))
    rb.fit(train_features, train[category])
    prediction = rb.predict_proba(test_features)[:,1]
    sub[category] = prediction

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate


In [30]:
scores = []
for train_target in categories:
    cv_score = np.mean(cross_val_score(rb, train_features, train[train_target], cv=3, scoring='roc_auc')) 
    scores.append(cv_score)
np.mean(scores)

0.8668138616107502

Точность получилась совсем не очень, подберем гиперпараметры на `GridCearchCV`, а также применим `OneVsRestClassifier`, так как имеем дело с miltilabel классификацией

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
  "estimator__n_estimators": [5, 50, 200],
  "estimator__max_depth" : [None, 10,20],
  "estimator__min_samples_split" : [2, 5, 10],
}
model_to_tune = OneVsRestClassifier(RandomForestClassifier(random_state=0,class_weight='auto'))
model_tuned = GridSearchCV(model_to_tune, param_grid=params, scoring='f1',n_jobs=2)

In [ ]:
for class_ in categories:
    y_this_class = (y_all_class==a_class)
    model_to_tune = RandomForestClassifier(random_state=0,class_weight='auto')
    model_tuned = GridSearchCV(model_to_tune, param_grid=params, scoring='f1',n_jobs=2)
    model_tuned.fit(train_features, train[y_this_class])

Также не стал запускать из-за плотной нехватки времени, но там все работает(можете проверить)

In [ ]:
rb = RandomForestClassifier(n_estimators=50, max_depth=20, min_samples_split=2)
for category in categories:
    print('... Processing {}'.format(category))
    rb.fit(train_features, train[category])
    prediction = rb.predict_proba(test_features)[:,1]
    sub[category] = prediction

`Random Forest` показал себя не с лучшей стороны,тк долго работает и точность выдает не самую наилучшую, даже после подбора параметров. Применим метод опорных векторов

In [31]:
from sklearn.svm import LinearSVC, SVC

In [38]:
clf = LinearSVC(C=0.7, max_iter=10000)
for category in categories:
    print('... Processing {}'.format(category))
    clf.fit(train_features, train[category])
    prediction = clf.predict(test_features)
    sub[category] = prediction

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate


In [39]:
from sklearn.calibration import CalibratedClassifierCV

Чтобы предсказать вероятность, а не класс требуется `CalibratedClassifierCV`

In [40]:
calib = CalibratedClassifierCV(base_estimator=clf)
for category in categories:
    print('... Processing {}'.format(category))
    calib.fit(train_features, train[category])
    prediction = calib.predict_proba(test_features)[:,1]
    sub[category] = prediction

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate


In [35]:
sub.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,150000,0.548476,0.003502,0.033639,0.002428,0.108342,0.004298
1,150001,0.019925,0.003077,0.012905,0.001776,0.011922,0.004098
2,150002,0.018510,0.002965,0.009980,0.001559,0.012460,0.003542
3,150003,0.001833,0.002349,0.003049,0.000978,0.004356,0.002659
4,150004,0.028122,0.001997,0.011492,0.001087,0.014278,0.003854


In [36]:
sub.to_csv('sub.csv', index=False)

In [43]:
scores = []
for train_target in categories:
    cv_score = np.mean(cross_val_score(calib, train_features, train[train_target], cv=3, scoring='roc_auc')) 
    scores.append(cv_score)
np.mean(scores)

0.9643494282892767

Данный метод показал себя с наилучшей стороны, особенно при параметре `C=0.7`, но к сожалению, лимит загрузок исчерпан)
 Кстати, что очень странно. При параметре равном 0.01, точность лучше на кросс валидации, однако при сабмите хуже, чем при 0.7

#### Embeddings

Попробуем другие методы для обработки, сначала сделаем необходимые импорты

In [ ]:
import tensorflow as tf
import string
import io
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook as tqdm
from gensim.models import FastText

Для начала сделаем функцию, чтобы можно было импортить любую эмбединг модель

In [ ]:
def vec(file):
    fin = io.open(file, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data
model_emb = vec('wiki-news-300d-1M.vec')

Сделаем эмбединг-словарь

In [ ]:
model_fast = {k: list(v) for k, v in model_emb.items()}

Создадим теперь мешок слов по приципу:<br>
Если слово встречается реже 100 раз, значит это очень специфичный термин,
используемый в очень маленьком количестве документов, либо же так называемая "опечатка", что маловероятно)

In [ ]:
count_vectorizer = CountVectorizer(min_df=100)
X_vect = count_vectorizer.fit_transform(X.apply(lambda x: ' '.join(w for w in x))).toarray()
X_vect

Составим эмбеддинги c 3 словами с наивысшим значением `tf_idf`. Если слова отсуствуют в эмбеддинг словаре, используем эмбеддинги следующих слов по значению `tf_idf`.<br>
На самом деле это оч ресурсозатратный процесс, даже кластер еле справляется, но один раз получилось это запустить. Точность была 92%, что ни в какие рамки не входит)

In [ ]:
tf_idf_dictionary = dict(zip(tf_idf_vectorizer.get_feature_names(), tf_idf_vectorizer.idf_))
X_emb_df = pd.DataFrame(columns=range(900))
for num in tqdm(range(len(train))):
    comment = X[num]
    words = [tf_idf_dictionary[word] for word in comment]
    important = np.array(comment)[np.argsort(words)[:-10:-1]]
    embeddings = np.array([])
    non_found = 0
    initial = 3
    count = 3
    for im in important[:count]:
        if im in model_fast: 
            embeddings = np.append(embeddings, np.array(model_fast[word]))
        else:
            non_found += 1
    while non_found != 0 and len(important)//300>count:
        im = important_words[count]
        if im_word in model_fast:
            embeddings = np.append(embeddings, np.array(model_fast[im_word]))
            non_found -= 1
        else:
            continue
        count += 1
    if len(embeddings) // 300 < initial:
        embeddings = np.append(embeddings, np.repeat(np.repeat(0, 300), initial-len(embeddings)//300))
    X_emb_df.loc[num] = np.ravel(embeddings)
    if (num+1) % 1000 == 0:
        X_emb_df.to_csv('DataFrame' + str(num-1000) + '-' + str(num) + '.csv')
        print('done')
        del X_emb_df
        X_emb_df = pd.DataFrame(columns=range(900))

Объединяем все в дф

In [ ]:
X_emb_df = pd.DataFrame(columns=map(str, range(900)))
dfs = sorted(filter(lambda x: x.startswith('DataFrame'), os.listdir()), key=lambda x: int(x[x.rfind('-')+1:-4]))
for one_df in tqdm(dfs):
    X_emb_df = X_emb_df.append(pd.read_csv(one_df).iloc[:, 1:])
    print(X_emb_df.shape)

Далее соединяем мешок слов и эмбединги

In [ ]:
X_ready = np.hstack((X_vect, X_emb_df.values))

Подводя итоги, хотелось бы сказать, что в данном случае самым эффективным оказался векторизатор `tf-idf`, а также лемматизация с помощью `nltk`. Токенизацию также можно сделать через векторизатор или вместе с лемматизацией `nltk`, тк точность от этого особо не меняется. Во всех случаях ведет себя по-разному.  Стопслова так чистятся во время векторизации. Самыми точными моделями оказались `Logistic Regression` и `Supported Vectors Machine`. Хотелось бы построить нейронку и добить точность до потолка, но это уже только в перспективе)